**FEDERATED LEARNING**

**Import relevant packages**

In [1]:
import numpy as np
import random
import cv2
import os
import pandas  as pd
import io
from google.colab import files
from imutils import paths
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from matplotlib import pyplot

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [2]:
uploaded = files.upload()
uploaded2 = files.upload()
uploaded3 = files.upload()
uploaded4 = files.upload()
uploaded5 = files.upload()
uploaded6 = files.upload()
uploaded7 = files.upload()

Saving d1_1m_0tm.csv to d1_1m_0tm.csv


Saving d2_1m_0tm.csv to d2_1m_0tm.csv


Saving d3_1m_0tm.csv to d3_1m_0tm.csv


Saving d4_1m_0tm.csv to d4_1m_0tm.csv


Saving d5_1m_0tm.csv to d5_1m_0tm.csv


Saving d6_1m_0tm.csv to d6_1m_0tm.csv


Saving d7_1m_0tm.csv to d7_1m_0tm.csv


In [3]:
df = pd.read_csv(io.BytesIO(uploaded['d1_1m_0tm.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded2['d2_1m_0tm.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded3['d3_1m_0tm.csv']))
df4 = pd.read_csv(io.BytesIO(uploaded4['d4_1m_0tm.csv']))
df5 = pd.read_csv(io.BytesIO(uploaded5['d5_1m_0tm.csv']))
df6 = pd.read_csv(io.BytesIO(uploaded6['d6_1m_0tm.csv']))
df7 = pd.read_csv(io.BytesIO(uploaded7['d7_1m_0tm.csv']))

card = len(df)
card2 = len(df2)
card3 = len(df3)
card4 = len(df4)
card5 = len(df5)
card6 = len(df6)
card7 = len(df7)
tot_card = card + card2 + card3 + card4 + card5 + card6 + card7

w1 = card/tot_card
w2 = card2/tot_card
w3 = card3/tot_card
w4 = card4/tot_card
w5 = card5/tot_card
w6 = card6/tot_card
w7 = card7/tot_card

cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]

cols2 = list(df2.columns)
cols2 = [cols2[-1]] + cols2[:-1]
df2 = df2[cols2]

cols3 = list(df3.columns)
cols3 = [cols3[-1]] + cols3[:-1]
df3 = df3[cols3]

cols4 = list(df4.columns)
cols4 = [cols4[-1]] + cols4[:-1]
df4 = df4[cols4]

cols5 = list(df5.columns)
cols5 = [cols5[-1]] + cols5[:-1]
df5 = df5[cols5]

cols6 = list(df6.columns)
cols6 = [cols6[-1]] + cols6[:-1]
df6 = df6[cols6]

cols7 = list(df7.columns)
cols7 = [cols7[-1]] + cols7[:-1]
df7 = df7[cols7]

# drop activity (last) column 
df = df.iloc[:,:-1]
df2 = df2.iloc[:,:-1]
df3 = df3.iloc[:,:-1]
df4 = df4.iloc[:,:-1]
df5 = df5.iloc[:,:-1]
df6 = df6.iloc[:,:-1]
df7 = df7.iloc[:,:-1]

# drop timestamp (first) column
df = df.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]
df6 = df6.iloc[: , 1:]
df7 = df7.iloc[: , 1:]

**Data preparation**

In [6]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X = list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x = sequence[i:end_ix]
		X.append(seq_x)
	return array(X)
 
values = df.values
values2 = df2.values
values3 = df3.values
values4 = df4.values
values5 = df5.values
values6 = df6.values
values7 = df7.values
# choose a number of time steps
n_steps = 16
# split into samples
X = split_sequence(values, n_steps)
X2 = split_sequence(values2, n_steps)
X3 = split_sequence(values3, n_steps)
X4 = split_sequence(values4, n_steps)
X5 = split_sequence(values5, n_steps)
X6 = split_sequence(values6, n_steps)
X7 = split_sequence(values7, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 29
X = X.reshape((X.shape[0], X.shape[1], n_features))
X2 = X2.reshape((X2.shape[0], X2.shape[1], n_features))
X3 = X3.reshape((X3.shape[0], X3.shape[1], n_features))
X4 = X4.reshape((X4.shape[0], X4.shape[1], n_features))
X5 = X5.reshape((X5.shape[0], X5.shape[1], n_features))
X6 = X6.reshape((X6.shape[0], X6.shape[1], n_features))
X7 = X7.reshape((X7.shape[0], X7.shape[1], n_features))

**Split data**

In [7]:
from sklearn.model_selection import train_test_split
def split_dataset(X):
    #we perform a 80:20 split of the dataset in conjunction with a randomization 
    #(shuffling the dataset is important to have a good training and a good generalization)
    X_train, X_test = train_test_split(X, test_size=0.20, random_state=42)
    return X_train, X_test

**Model building**

In [35]:
def model_creation(steps, features):
  k_size = 5
  model = keras.Sequential(
    [
        layers.Input(shape=(steps, features)),
        layers.Conv1D(filters=16, kernel_size=k_size, padding="same", strides=2, activation="relu"),
        #layers.Dropout(rate=0.2),
        layers.Conv1D(filters=8, kernel_size=k_size, padding="same", strides=2, activation="relu"),
        layers.Conv1DTranspose(filters=8, kernel_size=k_size, padding="same", strides=2, activation="relu"),
        #layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(filters=16, kernel_size=k_size, padding="same", strides=2, activation="relu"),
        layers.Conv1DTranspose(filters=29, kernel_size=k_size, padding="same"),
    ])
  return model

In [56]:
def model_fitting(model, train_data):
  model.fit(train_data, train_data, epochs=30, batch_size=256,validation_split=0.2,
            callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, mode="min")],
    )

In [57]:
X_train, X_test = split_dataset(X)
X_train2, X_test2 = split_dataset(X2)
X_train3, X_test3 = split_dataset(X3)
X_train4, X_test4 = split_dataset(X4)
X_train5, X_test5 = split_dataset(X5)
X_train6, X_test6 = split_dataset(X6)
X_train7, X_test7 = split_dataset(X7)

m1 = model_creation(n_steps, n_features)
m1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m1, X_train)

m2 = model_creation(n_steps, n_features)
m2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m2, X_train2)

m3 = model_creation(n_steps, n_features)
m3.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m3, X_train3)

m4 = model_creation(n_steps, n_features)
m4.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m4, X_train4)

m5 = model_creation(n_steps, n_features)
m5.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m5, X_train5)

m6 = model_creation(n_steps, n_features)
m6.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m6, X_train6)

m7 = model_creation(n_steps, n_features)
m7.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model_fitting(m7, X_train7)

#history = m1.fit(X_train, X_train, epochs=5, batch_size=128, validation_split=0.2,
#    callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, mode="min")],
#)

#m1.summary()
#pyplot.plot(history.history["loss"], label="Training Loss")
#pyplot.plot(history.history["val_loss"], label="Validation Loss")
#pyplot.legend()
#pyplot.show()

Epoch 1/30
47/47 [==============================] - 2s 21ms/step - loss: 0.1379 - val_loss: 0.0892
Epoch 2/30
47/47 [==============================] - 1s 17ms/step - loss: 0.0742 - val_loss: 0.0600
Epoch 3/30
47/47 [==============================] - 1s 17ms/step - loss: 0.0477 - val_loss: 0.0364
Epoch 4/30
47/47 [==============================] - 1s 17ms/step - loss: 0.0310 - val_loss: 0.0253
Epoch 5/30
47/47 [==============================] - 1s 16ms/step - loss: 0.0225 - val_loss: 0.0199
Epoch 6/30
47/47 [==============================] - 1s 16ms/step - loss: 0.0189 - val_loss: 0.0174
Epoch 7/30
47/47 [==============================] - 1s 17ms/step - loss: 0.0165 - val_loss: 0.0149
Epoch 8/30
47/47 [==============================] - 1s 17ms/step - loss: 0.0136 - val_loss: 0.0120
Epoch 9/30
47/47 [==============================] - 1s 16ms/step - loss: 0.0115 - val_loss: 0.0107
Epoch 10/30
47/47 [==============================] - 1s 17ms/step - loss: 0.0105 - val_loss: 0.0098
Epoch 11/

**Computing Average Weights**

In [58]:
weights1 = m1.get_weights()
weights2 = m2.get_weights()
weights3 = m3.get_weights()
weights4 = m4.get_weights()
weights5 = m5.get_weights()
weights6 = m6.get_weights()
weights7 = m7.get_weights()
lun = len(weights1)
#print("Pesi:", weights1)
#print("Pesi:", weights2)
avg_weights = list()

for i in range(lun):
    temp_weights = (weights1[i]*w1 + weights2[i]*w2 + weights3[i]*w3 + weights4[i]*w4 + weights5[i]*w5 + weights6[i]*w6 + weights7[i]*w7)
    avg_weights.append(temp_weights)

#print("avg_weights:", avg_weights)

global_model = model_creation(n_steps, n_features)
global_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
global_model.set_weights(avg_weights)
global_model.summary()
#dopo TEST

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 8, 16)             2336      
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 4, 8)              648       
_________________________________________________________________
conv1d_transpose_69 (Conv1DT (None, 8, 8)              328       
_________________________________________________________________
conv1d_transpose_70 (Conv1DT (None, 16, 16)            656       
_________________________________________________________________
conv1d_transpose_71 (Conv1DT (None, 16, 29)            2349      
Total params: 6,317
Trainable params: 6,317
Non-trainable params: 0
_________________________________________________________________


**Prepare test data and detect anomalies (446)**

In [12]:
from google.colab import files
uploaded_an = files.upload()

Saving merge_an_0tm.csv to merge_an_0tm.csv


In [13]:
from google.colab import files
uploaded_merge = files.upload()

Saving merge_0tm.csv to merge_0tm.csv


In [14]:
df_an = pd.read_csv(io.BytesIO(uploaded_an['merge_an_0tm.csv']))

cols_an = list(df_an.columns)
cols_an = [cols_an[-1]] + cols_an[:-1]
df_an = df_an[cols_an]

activity = df_an.iloc[:,-1]
cardinality = activity.shape[0]

anomaly_real = list()
for items in activity.iteritems():
  if 'ano' in items[1]:
    anomaly_real.append(items[0])

number_anomaly = len(anomaly_real)
number_ok = cardinality - number_anomaly
#for i in range(len(index)):
  #print(index[i])

# drop activity (last) column 
df_an = df_an.iloc[:,:-1]


"""
df_an['morning'] = 0
df_an['Dates'] = pd.to_datetime(df_an['timestamp']).dt.date
df_an['Time'] = pd.to_datetime(df_an['timestamp']).dt.time

date1 = time(18, 00, 00)
df_an.loc[df_an['Time'] >= date1, 'morning'] = 1
"""


# drop timestamp (first) column
df_an = df_an.iloc[: , 1:]
#df_an = df_an.iloc[:,:-1]
#df_an = df_an.iloc[:,:-1]

#print(df_an.head())

values_an = df_an.values

In [15]:
df_merge = pd.read_csv(io.BytesIO(uploaded_merge['merge_0tm.csv']))

cols_merge = list(df_merge.columns)

cols_merge = [cols_merge[-1]] + cols_merge[:-1]
df_merge = df_merge[cols_merge]

# drop activity (last) column
df_merge = df_merge.iloc[:,:-1]

# Order dataset by timestamp
#df['timestamp'] = pd.to_datetime(df['timestamp'])
#df['timestamp'].min(), df['timestamp'].max()


"""
df_merge['morning'] = 0
df_merge['Dates'] = pd.to_datetime(df_merge['timestamp']).dt.date
df_merge['Time'] = pd.to_datetime(df_merge['timestamp']).dt.time
  
date1 = time(18, 00, 00)
df_merge.loc[df_merge['Time'] >= date1, 'morning'] = 1
df_merge = df_merge.iloc[:,:-1]
df_merge = df_merge.iloc[:,:-1]
"""


# drop timestamp (first) column
df_merge = df_merge.iloc[: , 1:]

#aggiungere colonna attività e attività buona o anomala
print(df_merge.head())

values_merge = df_merge.values

X_merge = split_sequence(values_merge, n_steps)
X_merge = X_merge.reshape((X_merge.shape[0], X_merge.shape[1], n_features))
X_train_merge, X_test_merge = split_dataset(X_merge)

   wardrobe  tv  oven  ...  bathroomDoorLock  bathroomDoor  bathroomCarp
0         0   0     0  ...                 0             0             0
1         0   0     0  ...                 0             0             0
2         0   0     0  ...                 0             0             0
3         0   0     0  ...                 0             0             0
4         0   0     0  ...                 0             0             0

[5 rows x 29 columns]


**Threshold**

In [59]:
# Get train MAE loss
X_train_pred = global_model.predict(X_train_merge)
train_mae_loss = np.mean(np.abs(X_train_pred - X_train_merge), axis=1)

#pyplot.hist(train_mae_loss, bins=50)
#pyplot.xlabel("Train MAE loss")
#pyplot.ylabel("No of samples")
#pyplot.show()

# Get reconstruction loss threshold
threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)

Reconstruction error threshold:  1.0689212400466204


In [60]:
X_an = split_sequence(values_an, n_steps)

# Get test MAE loss.
X_an_pred = global_model.predict(X_an)
test_mae_loss_an = np.mean(np.abs(X_an_pred - X_an), axis=1)
#test_mae_loss_an = test_mae_loss_an.reshape((-1))

#pyplot.hist(test_mae_loss_an, bins=50)
#pyplot.xlabel("test MAE loss")
#pyplot.ylabel("No of samples")
#pyplot.show()

In [71]:
th = threshold*0.95
# Detect all the samples which are anomalies.
#anomalies = test_mae_loss_an > threshold
anomalies = test_mae_loss_an > th

print("Number of anomaly samples: ", np.sum(anomalies))
#print("Indices of anomaly samples: ", np.where(anomalies))

Number of anomaly samples:  176329


**Compute Confusion Matrix**

In [72]:
anomaly_detected = np.where(anomalies)[0]
#remove duplicates
anomaly_detected = list(dict.fromkeys(anomaly_detected))
TP = 0
FP = 0
TN = 0
FN = 0
for ind in anomaly_detected:
  if ind in anomaly_real:
    TP = TP+1
  else:
    FP = FP+1
for ind2 in anomaly_real:
  if ind2 not in anomaly_detected:
    FN = FN+1
TN = cardinality - TP - FP - FN 
print("TP:", TP, "\nTN:", TN, "\nFP:", FP, "\nFN:", FN)

TP: 34 
TN: 164548 
FP: 142665 
FN: 412


**Compute Indexes**

In [73]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

accuracy = (TP+TN)/cardinality
accuracy = truncate(accuracy, 3)

precision = TP/(TP+FP)
precision = truncate(precision, 3)

sensitivity = TP/(TP+FN)
sensitivity = truncate(sensitivity, 3)

specificity = TN/(TN+FP)
specificity = truncate(specificity, 3)

print("Accuracy:", accuracy, "\nPrecision:", precision, "\nSensitivity:", sensitivity, "\nSpecificity:", specificity)

Accuracy: 0.534 
Precision: 0.000 
Sensitivity: 0.076 
Specificity: 0.535
